<a href="https://colab.research.google.com/github/k2moon/bigbungi/blob/main/test/bigbungi_work02_regressor_01_ans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형 ▶ 제2유형 회귀(예측) 모델 수행하기
---


## 캐글 바이크 수요 예측

다음은 Bike Sharing Demand 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 대여량를 예측하고 해당 결과를 csv파일로 제출하시오.
(제출한 모델의 성능은 R^2 score 평가지표에 따라 채점)

* 결과 제출 양식 : PassengerId, Survived

- Kaggle: https://www.kaggle.com/competitions/bike-sharing-demand
- DataSet x_test : https://tinyurl.com/2fucdhv3
- DataSet x_train : https://tinyurl.com/4vy3adv5
- DataSet y_train : https://tinyurl.com/vr9m327a

- 계절 : 1 봄, 2 여름, 3 가을, 4 겨울
- 공휴일 : 1 공휴일, 0 평일
- 근무일 : 1 근무일, 0 휴일
- 날씨 : 1 아주 깨끗, 2 안개와 구름, 3 약간 눈 또는 비, 천둥 번개, 4 아주 많은 비 또는 우박

In [23]:
# 데이터 가져오기 read_csv()
import pandas as pd
import numpy as np

x_test = pd.read_csv("https://tinyurl.com/2fucdhv3", encoding='CP949')
# 시간당 자전거 대여량 데이터
x_train = pd.read_csv("https://tinyurl.com/4vy3adv5", encoding='CP949')
y_train = pd.read_csv("https://tinyurl.com/vr9m327a")

In [29]:
# 값 확인
pred[pred < 0]


array([], dtype=float64)

In [ ]:
# ------- 데이터 확인 및 EDA ---------

# 모든 컬럼값 확인 T
print(x_train.head(1).T)

# 행/열 확인 shape (생략가능)
print(x_train.shape)

# 요약정보 확인 info()
## 데이터타입, 결측치 등 확인, object 범주형
## object 범주형 인코딩 확인
print(x_train.info())
print(x_train['계절'].unique())

# 기초통계량 확인 describe()
## count, mean,std,min, 25%, 50%, 75%, max
## 데이터 스케일링 확인, 회귀, 비tree 계열
print(x_train.describe())
#print(X_train.describe().loc['min',:])

# 타깃값 확인 unique
## 1차원인지도 확인 (y,)
print(y_train['count'].value_counts())
y_train = y_train['count']
print(y_train.unique())

# EDA
data = pd.concat([x_train, y_train], axis=1)

## 계절 칼럼에 따른 대여 count 평균값
print(data.groupby(['계절'])['count'].mean())

## 공휴일 칼럼에 따른 대여 count 평균값
print(data.groupby(['공휴일'])['count'].mean())

## 근무일 칼럼에 따른 대여 count 평균값
print(data.groupby(['근무일'])['count'].mean())

## 날씨 칼럼에 따른 대여 count 평균값
print(data.groupby(['날씨'])['count'].mean())

# ------- 데이터 전처리 preprocessing ---------
## train, test 모두 처리 해야 함

# 결측치 처리하기 fillna()
## 임의로 결측치 삭제하지 말 것
## 평균, 중앙값, 상황에 따른 값

# 파생변수 생성
## 필요시 생성
## datetiem => 년, 월, 일, 시간, 요일
x_train['datetime'] = pd.to_datetime(x_train['datetime'])
print(x_train.info())

x_train['year'] = x_train['datetime'].dt.year
print(x_train['year'].unique())

x_train['month'] = x_train['datetime'].dt.month
print(x_train['month'].unique())

x_train['day'] = x_train['datetime'].dt.day
print(x_train['day'].unique())

x_train['hour'] = x_train['datetime'].dt.hour
print(x_train['hour'].unique())

# 0 월 ~ 6 일
x_train['dayofweek'] = x_train['datetime'].dt.dayofweek
print(x_train['dayofweek'].unique())

## EDA 
data = pd.concat([x_train, y_train], axis=1)

## year 칼럼에 따른 대여 count sum
print(data.groupby(['year'])['count'].sum())

## month 칼럼에 따른 대여 count 평균값
print(data.groupby(['month'])['count'].mean())
## 차이가 크지 않음
x_train = x_train.drop(columns = ['month'])

## day 칼럼에 따른 대여 count 평균값
print(data.groupby(['day'])['count'].mean())
## 차이가 크지 않음
x_train = x_train.drop(columns = ['day'])

## hour 칼럼에 따른 대여 count 평균값
print(data.groupby(['hour'])['count'].mean())

## dayofweek 칼럼에 따른 대여 count 평균값
print(data.groupby(['dayofweek'])['count'].mean())
## 차이가 크지 않음
x_train = x_train.drop(columns = ['dayofweek'])

# test 처리
x_test['datetime'] = pd.to_datetime(x_train['datetime'])
print(x_test.info())

x_test['year'] = x_test['datetime'].dt.year
print(x_test['year'].unique())

x_test['hour'] = x_test['datetime'].dt.month
print(x_test['hour'].unique())


# 불필요한 컬럭 삭제 drop(columns=[])
## PK 역할 컬럼 등 삭제
x_train = x_train.drop(columns = ['datetime'])
y_train = y_train.drop(columns = ['datetime'])


# 제출용 id 삭제전 저장
x_test_id = x_test['datetime']
x_test = x_test.drop(columns = ['datetime'])
print(x_train.isnull().sum())



# 범주형 변수 인코딩 objedt형
## 라벨 인코딩 LabelEncoder, 원핫인코딩  OneHotEncoder
## 라벨 인코딩(Tree 계열의 분류 알고리즘에 사용)



# 데이터 스케일링
## 연속형 변수의 최대, 최소 분포차가 클때
## MinMaxScaler, StandardScaler, RobustScaler
## test는 train용으로 transform()
## Tree 계열 필수는 아님

print(x_train.describe().T)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
print(x_train.describe().T)

# 상관관계 확인 corr()
## 필요 시 확인

# 전처리 확인 info()
print(x_train.info())
print(x_test.info())

# ------- 머신러닝 machine learning ---------

# 평가용 데이터 분리 model_selection
## stratify, stes_size, y값 1차원 확인
print(y_train.value_counts()/len(y_train))
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2)
print(Y_train.value_counts()/len(Y_train))

# 데이터 학습
## fit, predict, predict_proba
## 분류 XXXClassifier, LogisticRegression
## 회귀(예측) XXXRegressor(XXXRegression)
## 공통 ensemble
## RandomForestXXX : n_estimators (default = 100), max_depth, criterion
## criterion : 분할 품질을 측정하는 기능 ()
### RandomForestClassifier: criterion{“gini”, “entropy”, “log_loss”}, default=”gini”
### RandomForestRegressor: criterion{“mse”, “mae”}, / v1.2 criterion{“squared_error”, “absolute_error”}, default=”squared_error”
## GradientBoostingXXX : n_estimators, learning_rate(default =  0.1)

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


#model = DecisionTreeRegressor(random_state=42) 
#model = LinearRegression(random_state=42) 
model = RandomForestRegressor(random_state=42)  
#model = GradientBoostingClassifier(n_estmators = 100, max_depth=3) 

model.fit(X_train, Y_train)
pred = model.predict(X_test)


# 데이터 평가 metrics
## 회귀 : MAE, MSE, RMSE, R^2, RMSE는 np.sqrt(MSE값)
## 분류 : ROC_AUC, Accuracy(정확도), Precision(정밀도), Recall(재현율)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print('>>>>>> ', r2_score(Y_test, pred))


# ------- 답안 제출 ---------

# 전체학습데이터로 다시 학습
model.fit(x_train, y_train)

# 제출용 예측
pred = model.predict(x_test)


# 답안 제출 참고
# #아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
#pd.DataFrame({'cust_id': x_test_data.cust_id, 'gender': pred}).to_csv('424242.csv', index=False)

# 제출 최종 확인
#print(pd.read_csv("data/424242.csv"))